# Memory Networks

## Gated RNN memóriaproblémák

### Viszonylag kicsi "munkamemória"

Az LSTM és variánsainak memóriája (a hidden state mérete) meglehetősen kicsi

<img src="http://drive.google.com/uc?export=view&id=1-LbAhO8U_sfr5ipSaPTEm_0niPxS39u8" width="700px">

még nagy, mondjuk 2000-es réteghosszokkal és 64 bites lebegőpontos számokkal kalkulálva is csak kb. 128kbit = 16kB-ról beszélhetünk, ami már a PC-k őskorában sem számított soknak...

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Commodore_16_002a.png/1920px-Commodore_16_002a.png" width="700px">


#### A súlyok tárigénye

A súlyok száma viszont négyzetesen függ a memória méretétől, mivel a felejtő stb. kapuk sűrű rétegek.

Ha a rejtett állapot mérete $h$, akkor a kapuk súlyainak száma LSTM esetén $8 h^2$, vagyis a 2000-es példánál maradva

32 millió súlyról beszélhetünk, aminek a tárigénye már (szintén 64 bites számokkal számolva) kb. 256MB (!!).

#### Külső memória

Az alapkérdés tehát: hogyan tudnánk úgy növelni a munkamemóriát, hogy eközben a súlyok száma nem növekszik?

## Attention a seq2seq modellekben (emlékeztető)

Alap seq2seq architektúra:

<img src="http://suriyadeepan.github.io/img/seq2seq/seq2seq2.png">


A seq2seq feladatok jelentős része nem "holisztikus" abban az értelemben, hogy az output sorozat előállításának különböző fázisaiban az inputsorozat más más részei fontosak, más-más elemekre célszerű "figyelni", mások akár többé-kevésbé irrelevánsak is lehetnek. Ennek ellenére az alap decoder-encoder modell csak a teljes sorozatot reprezentáló, egyetlen végső rejtett állapot alapján dolgozik, nem fér hozzá az encoder korábbi rejtett állapotainak éppen releváns részéhez. Korábban bizonyos trükökkel próbálták "erősíteni" a korábbi állapotok hatását, pl. megfordítva adták be az inputot, vagy többször,de az igazi megoldásnak az ú.n. attention mechanizmus látszik:

<img src="https://cdn-images-1.medium.com/max/1600/0*SY3nv8-J6qX1GUxk.png">

Vagyis a dekóder minden egyes lépésben megkapja az előző rejtett állapot és kimenet mellett az enkóder összes rejtett állapotának egy _súlyozott átlagát_ is, mint kontextust. A kontextus a dekóder $i$. lépésében:

$$ c_i = \sum_{j=1}^{T}\alpha_{ij}h_j$$

ahol minden $h_k$ enkóder rejtett állapotra egy párhuzamosan tanított $A$ feed-forward háló ad egy 

$$e_{ik} = A(h_k, s_{i-1})$$ 

súlyt (a $h_k$ enkóder állapottal és $s_{i-1}$-gyel, vagyis a dekóder előző rejtett állapotával, mint inputtal), és ezekből az $\alpha_{ij}$ súlyok softmaxszal állnak elő:

$$\alpha_{ij} = \frac{\exp e_{ij}}{\sum_{k=1}^{T}\exp e_{ik}}$$


A klasszikus cikk az attention-mechanizmusról: [Bahdanau et al: "Neural machine translation by jointly learning to align and translate." (2014).](https://arxiv.org/pdf/1409.0473.pdf)

## Memory Networks

[Weston et al. (Facebook AI Research, 2014): Memory Networks](https://arxiv.org/pdf/1410.3916.pdf)

### Absztrakt architektúra: IGOR

- __Input__: Belső feature reprezentációkká alakítja a nyers inputot.
- __Generalizáció__: Az input alapján frissíti a memóriát, tipikusan tömöríteni/generalizálni próbál.
- __Output__: Új outputot generál (a feature reprezentációs térben) az input és az aktuális memóriaállapot alapján.
- __Response__: Az outputot a megkívánt külső formátumra hozza.

### A modulok kicsit részletesebben
#### Input
NLP alkalmazások esetében itt történhet a preprocessing és az embedding is.
#### Generalizáció
A legegyszerűbb megoldás egyszerűen eltárolja az aktuális input belső reprezentációját egy adott, az $x$ nyers inputtól függő memóriában:

$$m_{H(x)} = I(x) $$

ahol a $H(.)$ függvény határozza meg a megfelelő memóriát.

#### Output és response
- Tipikusan az output olvassa és állítja össze a releváns memóriákat (ez egyfajta reasoning-et is jelenthet)
- Az output alapján a response adja a végső választ, lehet pl. egy RNN dekóder.

#### Neural Memory Network Implementations: MEMNN

Egyszerű baseline megvalósítás: question answering (a memóriában tárolt tények alapján). Az input mindig egy mondat: vagy

- egy tényt rögzít, vagy 
- egy megválaszolandó kérdést tesz fel.

##### Generalizáció/tárolás

Egyszerűen mindig eltároljuk egy új slotban bejövő tényeket változtatás nélkül.

##### Output modul

$k$ számú releváns memóriatartalmat keres ki egy $S_o$ scoring függvény segítségével. Konkrétan $k=2$ esetén

$$o_1 = \mathrm{argmax}_{i=1\dots N}~ S_o(x, \mathbf{m}_i) $$

$$o_2 = \mathrm{argmax}_{i=1\dots N}~ S_o([x, \mathbf{m}_{o_1}], \mathbf{m}_i) $$

A végső output $[x, \mathbf{m}_{o_1}, \mathbf{m}_{o_2}]$, ez a response modul inputja.

##### Response

A legegyszerűbb esetben ez egyetlen szót generálunk válaszként, és ezt egy újabb scoring függvénnyel tesszük:

$$r = \mathrm{argmax}_{w \in W} ~ S_r([x, \mathbf{m}_{o_1}, \mathbf{m}_{o_2}], w) $$

##### Scoring függvények

Mind $S_o$, mind $S_r$ formája

$$S(x, y) = \phi_x(x)^\top U^\top U \phi_y(y)$$

ahol a $\phi$-k a szövegeket egy $D$ dimenziós feature space-be képzik (a legegyszerűbb modell bag-of-words reprezentációkat használt) az U-k pedig embedding mátrixok, tehát végeredményben a sűrű reprezentációk vetületét használjuk.

##### Training

A két embedding mátrixot, $U_o$-t és $U_r$-t próbáljuk optimalizálni SGD-vel, de nem end2end, hanem __strongly supervised__ módon, mivel a tanulóadat megjelöli a két releváns adatot is.

##### Javított modellek

- __Memória hashing:__ Ha túl sok a memóriában tárolt tény, akkor nagyon drága az összes tényt score-olni, ezért bucketekbe hasheljük a tényeket, és csak a kérdésnek megfelelő bucketekben keresünk. Pl. minden szóhoz tartozhat egy bucket, ekkor csak a kérdéssel közös szót tartalmazó tényeket vizsgáljuk. (Fejlettebb változat: clusterezzük a wordembeddingeket, és az egy-egy clusterhez tartozó szavakat tart. tények kerülnek egy bucketba).

- __Beérkezési idő:__ NLP feladatokhoz fontos reprezentálni azt is, hogy milyen sorrendben jöttek be az inputok, amelyeket score-olunk. A  $\phi$ featurereprezentációt kiegészítjük ilyen adatokkal.

- __Új szavak__: Ezeket a tipikus környezetükkel (BOW) modellezhetjük, ezt is hozzáadjuk a feature-ökhöz.

- __Output generálás__: teljes mondat RNN dekóderrel egyetlen szó helyett.

#### Eredmények

- __Large scale QA__ Keresés 14M memóriában tárolt (subject, relation, object) triplet között (pl. milne authored winnie-the-pooh) -- term. nyelvű kérdés alapján, pl. "Who is pooh's creator?"

> "The results show that MemNNs are a viable approach for large scale QA in
terms of performance."

- __Simulated World QA__ 

Kérdések megválaszolása egyszerű sztorik alapján.

>we also built a simple simulation of 4 characters, 3 objects and 5 rooms – with characters moving around, picking up and dropping objects. The actions are transcribed into text using a simple automated grammar, and labeled questions are generated in a similar way.

<img src="http://drive.google.com/uc?export=view&id=1oxe_-Wm4s4K-Ax880NCu4Z_lK-PFnriH">

<img src="http://drive.google.com/uc?export=view&id=1BtY9jKwtt3xr4NS9huadwCTllP-GQCkp" width="700px">

(Nehézség: Max. az utolsó előtti hányadik mondatban szerepelt utoljára a kérdezett tárgy. Actor vs actor + object kísérlet: az elsőben csak a "go" cselekvés volt megengedett, utóbbiban "get" és "drop" is)

<img src="https://memegenerator.net/img/instances/64071735/where-is-the-catch.jpg" width="400px"> 

### Térjünk vissza a tréninghez egy pillanatra...

ha kicsit részletesebben megnézzük, a következő loss-t használták:

<img src="http://drive.google.com/uc?export=view&id=1ZIjx9V0hmSpP1FfLAqNS_Wx399LeDzrp" width="700px">

Vagyis a loss egy konkrét példára az összes rossz jelöltre a jó jelölt score-jától való távolságnak a margintól való eltérése.
Rögzített jó jelöltek ($\mathbf{m_{o_1}, m_{o_1}}$) esetén ez (majdnem mindenütt) differenciálható $U_O$ és $U_R$ szerint, de ha ezek nem adottak, hanem a

$$o_1 = \mathrm{argmax}_{i=1\dots N}~ S_o(x, \mathbf{m}_i) $$

$$o_2 = \mathrm{argmax}_{i=1\dots N}~ S_o([x, \mathbf{m}_{o_1}], \mathbf{m}_i) $$

egyenleteknek megfelelően szerepeltetjük őket a loss-ban, akkor elveszítjük a differenciálhatóságot, mert a maximális pontszámú memória változásánál "ugrik" a függvény $\Rightarrow$ __a rendszer nem trénelhető end-to-end módon__. Ezen a problémán segít a...


## End-To-End Memory Networks (MemN2N)

[Sukhbaatar et al. (NYU,  Facebook AI Research, 2015): End-To-End Memory Networks](http://papers.nips.cc/paper/5846-end-to-end-memory-networks.pdf)

Koncepció: Adjuk egy folytonos, end-to-end változatát a Memory Network koncepciónak!

### Alapfeladat: QA

Megegyezik az eredeti MN alapfeladattal:

- Beérkezik input "tények" egy $x_1,\dots x_n$ sorozata és egy $q$ kérdés
- adjunk vissza egy $a$ választ a kérdésre.

A rendelkezésre álló dataset-ben csak a fenti adatok vannak megadva, tehát nincsenek megjelölve a releváns tények.

### Baseline modell

A memória egy rögzített hosszúságú buffer, az inputot ebben tároljuk. Két embedding mátrixot ($A$, $B$) használva mind a tényeket/memóriatartalmakat, mind a kérdést sűrű vektorreprezentációkká alakítjuk, és utána egyszerű koszinusztávolsággal pontozzuk az összes tény/memóriatartalom relevanciáját. Erre softmax-ot alkalmazva kapunk egy relevanciaeloszlást, aminek értéke minden $i$ memóriaindexre

$$p_i = \mathrm{Softmax}(u^\top m_i)$$

ahol $u$ a kérdés embedding vektora. 

Az $x_i$ tényekhez egy harmadik, $C$ embeddinggel $c_i$ outputvektorokat is rendelünk, és a response vektor egyszerűen ezek $p$ szerint súlyozott átlaga lesz:

$$o = \sum_{i}p_i c_i$$

A legegyszerűbb modell, amely egyszerűen egy egyszavas választ generál innen egyetlen feedforward réteggel és softmaxszal megy tovább:

$$a = \mathrm{Softmax}(W(o + u))$$

<img src="http://drive.google.com/uc?export=view&id=1O_toD6rZ-oJEnPjgXjcHkaAXGYjmZkWX" width="800px">

### Többrétegű architektúra

Minden rétegnek saját $A$ és $C$ embedding mátrixa van, és az első utáni rétegeket úgy helyezzük egymásra, hogy a $k + 1$. réteg egyedi vektorbemenete mindig az előző rétegre adott output:

$$u^{k+1} = u^k + o^k$$


<img src="http://drive.google.com/uc?export=view&id=1vq8UHIRn6ClmyLYWSAJiVzCNIoREmDQH" width="450px">

Kipróbált megoldások az $A$ és $C$ rétegek egymáshoz kapcsolására összesen $K$ réteg esetén:

- minden $k$-ra $A^{k+1}= C^k$ és $A^1 = B$, $W = C^K$

- RNN-szerű megoldás: minden $A_i$ és $C_i$ megegyezik -- ez lényegében egy speciális RNN architektúrának felel meg, amit fix lépésszámmal futtatunk.

#### Mondatinput embedding megoldások
- BOW: Szószintű embeddinget alkalmazunk, és a mondatvektor egyszerűen a mondat szóvektorainak összege (szórendfüggetlen).
- Positional Encoding (PE): Mondatpozíciótól függően súlyozzuk a szóvektorokat az összegben (szórendfüggő).

### Eredmények

#### A bAbI dataset

bAbI feladatok: szintetikus, szimulációval előállított QA "játék" dataset

([Weston et al (Facebook AI Research, 2016): Towards ai-complete question answering: A set of prerequisite toy tasks.](https://arxiv.org/pdf/1502.05698.pdf)):

>All of the tasks are noiseless and a human able to read that language can potentially achieve 100%
accuracy. We tried to choose tasks that are natural to a human: they are based on simple usual situations and no background in areas such as formal semantics, machine learning, logic or knowledge
representation is required for an adult to solve them.

>The data itself is produced using a simple simulation of characters and objects moving around and
interacting in locations, described in Section 4.  The simulation allows us to generate data in many
different scenarios where the true labels are known by grounding to the simulation.

Komponensek:

- "entitások"
  - helyek
  - tárgyak
  - személyek
- állapotok
  - abszolút/relatív hely
  - mentális állapot
- tulajdonságok
  - méret
  - szín
- cselekedetek:
  - go _location_, get _object_, get _object1_ from _object2_, put _object1_ in/on _object2_, give _object_ to
_actor_, drop _object_, set _entitity_ _state_, look, inventory and examine _object_.

"For each task, we describe it by giving a small sample of the dataset including statements, questions and the true
labels (in red) in Tables 1 and 2."

<img src="http://drive.google.com/uc?export=view&id=1zwM3eG-QuTkcyWWFzgius2r_xCMyTPOo" width="700px">

<img src="http://drive.google.com/uc?export=view&id=1GpyNrRy9B294D01SpspKX9mpxVFqp0D5" width="700px">




#### N2N MN eredmények a bAbI datasetten


- Az N2N MN jelentősen megverte a baseline LSTM-et (51,3 vs 12,4% mean error)
- Az N2N MN legerősebb beállításokkal csak összemérhető az erősen felügyelt MN-nel (12,4 vs 6,7% mean error)

## Dynamic Memory Networks (DMN)

[Kumar et al. (2016): Ask me Anything: Dynamic Memory Networks for Natural Language Processing](http://proceedings.mlr.press/v48/kumar16.pdf)

__Célkitűzés:__ Általános kérdésmegválaszoló framework.

A legtöbb NLP-s feladat átfogalmazható kérdésmegválaszolási feladattá:

#### Klasszikus kérdésmegválaszolás:
_Bemenő információk (tények):_
- Jane went to the hallway.
- Mary walked to the bathroom.
- Sandra went to the garden.
- Daniel went back to the garden.
- Sandra took the milk there.

_Kérdés:_ Where is the milk?

_Válasz:_ garden

#### Sentiment analysis/POS tagging

_Bemenő információ_: It started boring, but then it got interesting.

_Kérdés_: What’s the sentiment?/What are the POS tags?

_Válasz_: positive/PRP VBD JJ , CC RB PRP VBD JJ

### Moduláris architektúra

<img src="http://drive.google.com/uc?export=view&id=1rJKn4sHmtcTzipr91t4macxy9jpD7pC2" width="500px">

- __Input modul:__ A bemenő szöveget, amely lehet egy mondat, de akár több dokumentum is, sűrű vektorreprezentációk sorozatává alakítja.

- __Kérdés modul:__ A kérdést alakítja egyetlen sűrű vektorrá.

- __Epizodikus memória:__ Egy attention mechanizmus segítségével előállít egy "releváns tények" vektorreprezentációt.

- __Válasz modul:__ Előállítja a választ az epizodikus memória és a kérdésvektor alapján.

### A modulok részletesen

#### Input modul

- A  tokenizált inputot először egy (a kérdésmodullal közös) word embedding segítségével szóvektorsorozattá alakítjuk
- A szóvektorsorozatot egy RNN (GRU/LSTM) hidden-state vektorok sorozatává alakítja
  - ha egy mondat a bemenet (pl. POS tagging), akkor minden tokenhez tartozik egy hidden state
  - ha több, akkor a mondatok végén fennálló hidden state-ekből áll a reprezentáció

#### Kérdés modul

- A felépítése megegyezik az input modullal, de az output egyszerűen a kérdés feldolgozása végén adódó hidden state.


#### Epizodikus memória
<img src="http://drive.google.com/uc?export=view&id=1-1yrOVyHXZiqwHb4I-kdGXOWtlTe4C7b">

Az epizodikus memória egy vagy több menetben összegzi az összes input "tényt" úgy, hogy egy attention mechanizmus minden tényhez egy súlyt rendel.


##### Az attention súlyok kiszámítása

A súlyokat egy feed-forward háló számítja ki a következő paraméterek alapján:

- $c$: a tényvektor, amihez a súlyt keressük
- $m$: a memória állapota az előző menet végén
- $q$: a kérdésvektor

ezekből először előállítjuk a köv. $z(c, m, q)$ hasonlósági feature vektort:

$$z(c, m, q) = [c, m, q, c\circ q, c\circ m, |c-q|, |c-m|, c^\top W^{(b)}q,c^\top W^{(b)}m]$$

majd ezzel az inputtal a feed-forward hálóval kiszámítjuk a súlyt:

$$G(c, m, q)=\sigma(W^{(2)}\tanh(W^{(1)}z(c, m,q) + b^{(1)}) + b^{(2)})$$

Vannak datasettek, amelyek megadják a tényrelevanciát is (pl. [Facebook bAbI](https://research.fb.com/downloads/babi/)), ott ez a komponens supervised módon cross-entropy losssal tanítható, egyébként marad az end-to-end.



##### A memória update menete és vége

Az $i.$ frissítés során az összes tényen végighalad egy RNN (konkrétan GRU) a köv. belső állapot egyenlettel:

$$h_t^i = g^i_t RNN(c_t, h^i_{t-1}) + (1-g^i_t)h^i_{t-1}$$

Az RNN kezdő belső állapota a $q$ kérdésvektor, és az $i.$ memóriaállapot nem más, mint az RNN sorozatvégi belső/rejtett állapota.

_Megállás_: Az input végéhez adunk egy speciális STOP szimbólumot, és ha az attention ezt választja, akkor megállunk. (Ha nincs supervised adat a megállásról, akkor egyszerűen egy fix maximum menetszámot használunk.)


##### Válasz modul

A válaszmodul feladatfüggően vagy minden memórialépésnél (pl. POS-tagging), vagy egy teljes memóriafrissítés végén aktiválódik.

Itt is egy RNN-t (GRU)-t használunk, amelyet a $q$ kérdésvektorral inicializálunk, és kimenete minden $t$ time stepnél az $a_t$ belső állapotból

$$y_t = \mathrm{softmax}(W^{(a)}a_t)$$

belső állapota pedig

$$a_t =RNN([y_{t-1}, q], a_{t-1})$$

Fontos, hogy ha több teljes memóriafrissítést végzünk, akkor több választ/válaszsorozatot kapunk.


### Eredmények

Megjavította a state of the artot

- Question Answering-ben (93.3 $\Rightarrow$ 93,6% a bAbI datasetten)
- POS tagging (WSJ PTB: 97,5 $\Rightarrow$ 97,56%)
- Sentiment Analysis (Stanford Sentiment Bank: 87,8 $\Rightarrow$ 88,6%)

### Az epizodikus memória működése

#### Hány menetben érte el a legjobb eredényt a rendszer


<img src="http://drive.google.com/uc?export=view&id=1SA7GQJ_9_YgLNDBI77buS1LRQP671mVI" width="400px">

#### Attention vizualizációk

<img src="http://drive.google.com/uc?export=view&id=1PYjbNHlytpsoL8P3IGw2VIY1rUh0929i" width="800px">

## Neurális Turing-gépek

[Graves et al. (Google DeepMind, 2014): Neural Turing Machines](https://arxiv.org/pdf/1410.5401.pdf)

### A kiindulás: a Turing-gép (1937)

[Turing (1937): On Computable Numbers](http://l3d.cs.colorado.edu/~ctg/classes/lib/canon/turing-compnum.pdf)


<img src="http://drive.google.com/uc?export=view&id=1lLi50m3zJSHCcS0gZ3z8SDOx5D-2rWzF" width="500px">

A Turing-gépeket a (végtelen) írható külső memóriájuk teszi lényegesen erősebbé a véges állapotú automatáknál:

#### Turing-gép vs FSA különbségek:

- A Turing gép írni és olvasni is tudja a külső memóriát, az FSA csak olvas.
- A Turing gép feje tetszőleges irányban tud mozogni a szalagon, az FSA sorban, egyirányban olvassa az inputot és nem tud visszafelé haladni.
- A Turing gép szalagja végtelen, az FSA inputja és outputja véges.

Egy (a kiszámított függvényekre nézve természetesen ekvivalens) Turing-gép variáció input és output szalagokkal:

<img src="https://www.staff.ncl.ac.uk/joel.wallenberg/ContextsJoelGeoff/lectur14.gif" width="600px">

#### Church-Turing tézis

> Ha egy számítási feladat végrehajtható mechanikusan, egy előre megadott algoritmust végrahajtva, akkor megoldható egy Turing-gép segítségével is.

#### Kapcsolat a Turing-gépek és neurális hálók között:

Mint [Siegelmann and Sontag (1995). On the computational power of neural nets](http://research.cs.queensu.ca/home/akl/cisc879/papers/SELECTED_PAPERS_FROM_VARIOUS_SOURCES/05070215382317071.pdf)
kimutatta, tetszőleges Turing gép szimulálható RNN-ekkel.

### Alapötlet
Próbáljuk meg a fenti architektúrát úgy megvalósítani, hogy a kontroller és az író/olvasó feje(ek) szerepét egy neurális háló játssza!

### Architektúra

<img src="http://drive.google.com/uc?export=view&id=1J9-WcipTgqpQfUMIpcfFwfbctUyzLbvq" width="500px">

Látni fogjuk, hogy a Turing-gépen kívül a modern, CPU-alapú számítőgéparchitektúra is inspirációt jelentett:

<img src="https://sanjayachauwal.files.wordpress.com/2017/10/overall.gif?w=590&h=345&crop=1" width="500px">

A "controller" és az "író/olvasó" fejek tulajdonképpen a CPU szerepét játsszák.


### Alapmodell

#### Külső memória ("Memory Bank")

Egyszerűen egy $N\times M$ mátrix, ahol

- $N$ a memóriahelyek száma
- $M$ az egyetlen memóriahelyen tárolható vektor hossza.

#### A memória olvasása

A MemN2N-hez hasonlóan egy adott $t$ pillanatban egy attention-szerű $\mathbf{w}_t$ súlyozással olvassuk a memóriát -- a súlyozásokat az olvasófej produkálja.
A súlyok normalizáltak, tehát minden $\mathbf{w}_t(i)$ vektorkoordinátára

$$\mathbf{w}_t(i)\in [0, 1]$$

és

$$\sum_{i=0\dots N-1} \mathbf{w}_t(i) = 1$$

a kiolvasott memóriatartalom, a $t$ időponthoz tartozó "read vector" egyszerűn az összes tárolt vektor súlyozott összege, tehát

$$\mathbf{r}_t = \sum_{i=0\dots N-1} \mathbf{w}_t(i)\mathbf M_t(i)$$




#### A memória írása

Az LSTM-ekhez hasonlóan az írás két fázisban történik, egy _törlés_ és egy _hozzáadás_ operációból áll.
Ezeket az írófej a következő paraméterekkel határozza meg:

- Az olvasáshoz hasonlóan egy normalizált $\mathbf{w}_t$ súlyozás
- Egy $M$ hosszú $\mathbf{e}_t$ "törlésvektor", ahol minden koordináta $\in [0, 1]$
- Egy $M$ hosszú $\mathbf{a}_t$ "hozzáadásvektor"

Maguk a műveletek:

- Törlés: $\hat{\mathbf M}_t(i) \leftarrow \mathbf M_t(i)[\mathbf 1  - \mathbf{w}_t(i)\mathbf e_t]$
- Hozzáadás: ${\mathbf M}_t(i) \leftarrow \hat{\mathbf{M}}_t(i) + \mathbf{w}_t(i)\mathbf a_t$

Röviden: egy "elmosódott címzéssel" (a súlyozással) hajtunk végre a címzett memóriatartalmakon az LSTM "forget" és "update" műveleteivel analóg műveleteket.

#### Címzés: hogyan állnak elő az írási/olvasási súlyozások?

Két különböző fajta címzést, illetve ezek kombinációját képes használni a rendszer:

##### "Content-based addressing"

Itt a MemN2N-hez hasonlóan egy hasonlósági mérték szerint súlyozunk:

Az írófej produkál egy $M$ hosszú $\mathbf k_t$ "kulcsvektort", ehhez a kulcshoz való hasonlóság szerint súlyozunk, a  $K[\cdot,\cdot]$ hasonlósági metrika ás $\beta_t$ "kulcserősség" szerint:

$$\mathbf w_t^c(i)= \frac{\exp(\beta_t K[\mathbf k_t, \mathbf M_t(i)])}{\sum_j \exp(\beta_t K[\mathbf k_t, \mathbf M_t(j)])}$$

Vagyis $\mathbf w_t$ egy softmax a $\beta_t K[\cdot, \cdot]$ hasonlóságok fölött -- a konkrét implementációban $K$ a koszinusztávolság.

##### "Location-based addressing"

Ez a memóriában előre/hátraugrást valósít meg, tehát egy adott súlyozást "shiftel" megadott módon.

Konkrétan, az eltolást egy $\mathbf s_t$ "shift vektor" határozza meg, amely egy eloszlás a lehetséges egész shiftek fölött:

<img src="http://drive.google.com/uc?export=view&id=1-esaoX3CrgPvkMDRi4jGogcdwYeEvCpj" width="700px">


Mivel az így kapott súlyozás esetleg túl "életlen" lehet, ha $\mathbf s_t$ túl egyenletes, ezért a végső súlyt egy $\gamma_t > 1$ "élesítéssel" kapjuk meg:

$$w_t(i) \leftarrow \frac{\tilde{w}_t(i)^{\gamma_t}}{\sum_j{\tilde{w}_t(j)^{\gamma_t}}}$$ 


##### A két címzési típus kombinációja
A fent leírt eltolási műveletet az adott fej által az aktuális $t$-ben produkált tartalom-alapú súlyozás és a $t-1$ lépésben használt súlyozás egy $g_t \in (0, 1)$ paraméter szerinti lineáris kombinációjára alkalmazzuk:

$$
\mathbf w^g_t = g_t\mathbf w^c_t + (1-g_t)\mathbf w_{t-1}
$$

A címzés teljes működése ennek megfelelően:


<img src="http://drive.google.com/uc?export=view&id=1J5UXT-k0x25We8DoM69kJY5QU8uLz-sW" width="800px">


### A teljes rendszer működése 

<img src="http://drive.google.com/uc?export=view&id=1uB1WjSkWzXttN9Dx9riDe_H8Hb7Z-op9" width="400px">
- Mind író, mind olvasó fejből több lehet.
- Az írási műveletek (a törlés, ás a hozzáadás) kommutatívak, tehát ha több írófej van a rendszerben, akkor mindegy, milyen sorrendben hajtják végre az egyes fejek ezeket a műveleteket.
- A kontroller lehet feedforward vagy RNN háló -- a feedforward kontroller szimulálhatja a rekurrens működést úgy, hogy minden időlépésben olvas és ír egy adott memóriahelyet.

### Kísérletek

- Három architektúrával:
  - NTM + feedforward controller
  - NTM + LSTM controller
  - LSTM
  
#### Copy
Az input egy random sorozat egy delimiterrel a végén --  a feladat ennek az outputként való visszaadása.

A trénelés csak 1--20 hosszúságú sorozatokkal történt, de a NTM-ek (szemben az LSTM)-mel generalizáltak kb. a köv. algoritmust megtanulva:

```
initialise: move head to start location
while input delimiter not seen do
  receive input vector
  write input to head location
  increment head location by 1
end while
return head to start location
while true do
  read output vector from head location
  emit output
  increment head location by 1
end while
```
#### Repeat Copy

Az input egy random sorozat és egy ismétlésszám: az outputnak az ismétlésszámszor kell tartalmazni a sorozatot.
A sorozathossz generalizációban itt is sokkal jobb volt az MTM, de az ismétlésszámmal problémái voltak

#### Associative Recall

Az input egy delimitált sorozatokból álló lista, a lista egy random eleme, a rendszernek a lista köv. elemét kell visszaadni.

#### Dynamic N-Grams

Tud-e n-gram modellt építeni? A feladat az, hogy megjósolja egy (bináris) sorozat folytatását. 200 hosszúságú sorozatok köv. elemét kellett megjósolni.

#### Priority sort

Input: random vektorsorozat a vektorokhoz rendelt prioritásokkal, a feladat a prioritás szerinti rendezés.


## Neural Turing Machine 2.0: Differentiable Neural Computer

[Graves et al (Google DeepMind, 2016): Hybrid computing using a neural
network with dynamic external memory](https://www.dropbox.com/s/0a40xi702grx3dq/2016-graves.pdf)

### Architekturális újítások/változtatások
- Egy írófej több olvasófej
- Az írófej csak content/kulcs alapú címzést használ
- Egy külön mátrix tárolja (simítottan), hogy mely memóriacímek kerültek egymás után írásra
- Egy használtsági vektor tárolja, hogy mely memóriacímeket használta már a rendszer
- Az olvasás három különböző módon történhet:
  - content/kulcs alapon
  - a korábbi írási sorrendek szerint előre vagy hátra lépve
  - kiválasztva egy üres (kevésbé/régebben használt) memóriahelyet
  
<img src="http://drive.google.com/uc?export=view&id=1HQMkHgWYUL348DT86ZQe371snij1Ui6X">
  




### Alkalmazások/eredmények

#### bAbI

Mean test error rate 7.5% $\rightarrow$ 3.8%

#### Véletlenszerűen generált gráffeladatok

<img src="http://drive.google.com/uc?export=view&id=1ckenwxkD75EwfRJSED0O0xi20PtCEMJ3" width="700px">

<img src="http://drive.google.com/uc?export=view&id=1wr7CT728KYoVG44P89R5WnVmUVAr5WLC"  width="600px">

#### Objektummozgatás (mini SHRDLU)

Itt már a Reinforcement Learning területén járunk...


## Természetesen a történetnek nincs vége...

- Architekturális újítások, pl.: 
  - [Dulcehre et al (2017): Dynamic Neural Turing Machine with Soft and Hard Addressing Schemes](https://arxiv.org/pdf/1607.00036.pdf)
    (Tanulható memóriacímeket vezet be, plusz SGD helyett a diszkrét attentionös variánsban REINFORCE nevű (vajon honnan származó...) eljárással optimalizál.)
- Alkalmazások, különösen Memory Networkökre, pl.
  - [Chen et al (2018): Sequential Recommendation with User Memory Networks](http://xu-chen.com/resources/paper-pdf/sequential-rec-memory-network-wsdm18.pdf)
  - [Xiong et al (2016: Dynamic Memory Networks for Visual and Textual Question Answering](https://arxiv.org/pdf/1603.01417)

# Implementációk

- [MemN2N TF-ben](https://github.com/domluna/memn2n)
- [Dynamic Memory Networks TF-ben](https://github.com/barronalex/Dynamic-Memory-Networks-in-TensorFlow)
- [Differentiable Neural Computer TF-ben a Google-től](https://github.com/deepmind/dnc)

# Hogyan kövessük hatékonyan hogy mi folyik AI téren?

(Továbbiakban kutatási publikáció = "paper" vagy "papír").

## Probléma 1.: Össz mennyiség / sebesség

<img src="https://thumbor.forbes.com/thumbor/960x0/https%3A%2F%2Fblogs-images.forbes.com%2Flouiscolumbus%2Ffiles%2F2018%2F01%2Fannually-published-papers.jpg">

Bővebben [itt](https://www.forbes.com/sites/louiscolumbus/2018/01/12/10-charts-that-will-change-your-perspective-on-artificial-intelligences-growth/#384b45647583)

### Ráadásul Open Science

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/ArXiv_web.svg/500px-ArXiv_web.svg.png">

**A Forrás: [arXiv](https://arxiv.org/)** 

## Megoldás: Használj szűrőt!

### [Arxiv Sanity preserver]()

Személyre szabott recommender engine az ArXiv böngészéséhez. (AI-kereső-AI-kereső-AI... :-P


<img src="https://camo.githubusercontent.com/cd467093cba639a7fc914e369f47ecad0ca16ded/68747470733a2f2f7261772e6769746875622e636f6d2f6b617270617468792f61727869762d73616e6974792d7072657365727665722f6d61737465722f75692e6a706567">


Bemutató [itt](https://www.youtube.com/watch?v=S2GY3gh6qC8&feature=youtu.be)


### Közösség, mint szűrő

- Kövess influencer-eket Twitteren, Facebookon vagy LinkedIn-en
  - Vagy "nagy" név alapján (Hinton, LeCun, Bengio, Ng, Schmidhuber, Hochreiter, Sutskever, Goodfellow, Larochelle, Karpathy, ... ...)
  - Akinek a cikkét olvastad és értékesnek találtad
  - Vagy listából (nem olyan jók, cserébe van sok :-)
- "[A Facebook csoport"](https://www.facebook.com/groups/DeepNetGroup/) (137k members and counting...) 

## Probléma 2.: Egységnyi papír komplexitása

<img src="https://cdn-images-1.medium.com/max/1400/1*OlOpTJNL3Zyf6MOaZO05lw.jpeg">

## Megoldás: Olvasási stratégia (Szubjektív javaslat)

Jó megközelítés a komplex AI papírok olvasásához [itt](https://medium.com/ai-saturdays/how-to-read-academic-papers-without-freaking-out-3f7ef43a070f).

<img src="https://askabiologist.asu.edu/sites/default/files/resources/articles/anatomy_of_an_article/anatomy_paper_diagram3.jpg">

** I. "A buszon" **
1. Olvasd el a címet (ha nem érdekes, stop)
2. Olvasd el az abstractot (ha nem érdekes, stop)
3. Lapozz a Result részhez, keresd és értelmezd a táblázatos eredményeket (ha nem érdekes, stop)
4. Gyűjtsd olvasólistába.

** II. "Amikor van időd" **
5. Frissítsd fel az abstract és results részt
6. Olvasd át és próbáld megérteni a methods részt
  - Mit nem ír le, mit nem értesz?
  - Mi a "common assumption"?
7. Tárd fel a "common assumption" elemeket a referencia jegyzék segítségevel. (Kövesd vissza "A Nagy Papírokig")
8. Jegyzetelj
9. Nézd meg a konferencia vagy az adott előadó videóját! (Ha van...)
10. Keress implementációt
11. Kezdd újra az 6.-tól amíg jó nem lesz!
